In [ ]:
from sys import stdin

input = stdin.readline
d = [(1, 0), (-1, 0), (0, 1), (0, -1)]
d2 = [(-1, -1), (1, 1), (-1, 1), (1, -1), (-1, 0), (1, 0), (0, -1), (0, 1)]


def solve():
    N, M = map(int, input().split())
    board = []
    virus = []
    safeArea = []

    for i in range(N):
        board.append(list(map(int, input().split())))
        for j in range(len(board[i])):
            if board[i][j] == 2:
                virus.append((i, j))
            elif board[i][j] == 0:
                safeArea.append((i, j))

    def nearWall(_x, _y, _board):
        l, r, u, d = False, False, False, False
        for dx, dy in d2:
            x, y = _x + dx, _y + dy
            if x < 0 or (x < _x and 0 <= y < M and _board[x][y] == 1):
                u = True
            elif x >= N or (x > _x and 0 <= y < M and _board[x][y] == 1):
                d = True
            if y < 0 or (y < _y and 0 <= x < N and _board[x][y] == 1):
                l = True
            elif y >= M or (y > _y and 0 <= x < N and _board[x][y] == 1):
                r = True
            if (l and r) or (u and d):
                return True

        return False

    def countSafeCell(a, b, c):
        _board = [[*board[i]] for i in range(len(board))]
        _board[a[0]][a[1]] = 1
        _board[b[0]][b[1]] = 1
        _board[c[0]][c[1]] = 1

        countSafeArea = len(safeArea) - 3
        if (nearWall(*a, _board) and nearWall(*b, _board) and nearWall(*c, _board)):

            q = [*virus]
            idx = 0
            while idx != len(q):
                _x, _y = q[idx]
                idx += 1
                for dx, dy in d:
                    x, y = _x + dx, _y + dy
                    if 0 <= x < N and 0 <= y < M and _board[x][y] == 0:
                        q.append((x, y))
                        _board[x][y] = 2
                        countSafeArea -= 1
            return countSafeArea

        return 0

    def buildWall():
        _ans = 0
        for i in range(len(safeArea)):
            for j in range(i + 1, len(safeArea)):
                for k in range(j + 1, len(safeArea)):
                    _ans = max(_ans, countSafeCell(safeArea[i], safeArea[j], safeArea[k]))
        return _ans

    return buildWall()


print(solve())

In [ ]:
# Problems from : https://www.acmicpc.net/problem/14502
# Samsung SW 역량 테스트


def get_input():
    N = [int(n) for n in input().split(' ')[:2]]
    _map = list()
    for i in range(N[0]):
        _map.append([int(x) for x in input().split(' ')[:N[1]]])
    return N[0], N[1], _map

def spreadAndCount(_map):

    def count_safe(_map):
        count = (len(_map)) * (len(_map[0]))
        for row in _map:
            count = count - row.count(1) - row.count(2)
        return count

    result = [x[:] for x in _map]
    N = len(_map)
    M = len(_map[0])

    queue = list()
    for i in range(1, N - 1):
        for j in range(1, M - 1):
            if result[i][j] == 2:
                queue.append((i,j))

    while(True):
        new_queue = []
        for i, x in enumerate(queue):
            i, j = x[0], x[1]
            if result[i-1][j] not in (1, 2):
                result[i - 1][j] = 2
                new_queue.append((i-1,j))
            if result[i+1][j] not in (1, 2):
                result[i + 1][j] = 2
                new_queue.append((i+1,j))
            if result[i][j-1] not in (1, 2):
                result[i][j-1] = 2
                new_queue.append((i,j-1))
            if result[i][j+1] not in (1, 2):
                result[i][j+1] = 2
                new_queue.append((i,j+1))
        queue = new_queue
        if not queue:
            return count_safe(result)


def set_wall(_map, num=1):
    def check_8neighbor(inp_map, n, m):  # 8 neighbor
        score = 0
        for a in range(n - 1, n + 2):
            for b in range(m - 1, m + 2):
                if inp_map[a][b] == 1:
                    score += 1
        if score >= 1:
            return True
        else:
            return False

    N = len(_map)
    M = len(_map[0])
    crnt_map = [x[:] for x in _map]  # deep copy

    maximum = 0
    count = 0
    # set candidate
    candidate = list()
    for n in range(1, N-1):
        for m in range(1, M-1):
            if crnt_map[n][m] == 0 and check_8neighbor(crnt_map, n, m):
                candidate.append((n, m))

    for w in candidate:
        crnt_map[w[0]][w[1]] = 1  # set wall
        if num == 1:
            count = set_wall(crnt_map, num + 1)
            crnt_map[w[0]][w[1]] = 3  # 지나갔던 곳
        if num == 2:
            count = set_wall(crnt_map, num + 1)
            crnt_map[w[0]][w[1]] = 4  # 지나갔던 곳
        if num == 3:
            count = spreadAndCount(crnt_map)  # spread the plague And measure the area of safe zone
            crnt_map[w[0]][w[1]] = 5  # 지나갔던 곳
            if maximum < count:
                maximum = count
            continue

        if maximum < count:
            maximum = count

    return maximum


def isolate(_map):
    M = len(_map[0])
    # padding
    inp_map = [[1 for _ in range(M)]] + _map
    for i, m in enumerate(inp_map):
        inp_map[i] = [1] + m + [1];
    inp_map.append([1 for _ in range(M+2)])

    res = set_wall(inp_map)  # dynamic programming, DFS

    return res


if __name__ == "__main__":
    N, M, _map = get_input()
    result = isolate(_map)
    print(result)